In [5]:
import pandas as pd
from io import StringIO

df = pd.read_csv('data.csv')

col = ['URLs','Headline','Body','Label']
df = df[col]
df = df[pd.notnull(df['Body'])]
df.columns = ['URLs','Headline','Body','Label']

print(df)

                                                   URLs  \
0     http://www.bbc.com/news/world-us-canada-414191...   
1     https://www.reuters.com/article/us-filmfestiva...   
2     https://www.nytimes.com/2017/10/09/us/politics...   
3     https://www.reuters.com/article/us-mexico-oil-...   
4     http://www.cnn.com/videos/cnnmoney/2017/10/08/...   
5     http://beforeitsnews.com/sports/2017/09/jetnat...   
6     https://www.nytimes.com/2017/10/10/us/politics...   
7     https://www.reuters.com/article/us-india-cenba...   
8     https://www.reuters.com/article/us-climatechan...   
9     https://www.reuters.com/article/us-air-berlin-...   
10    https://www.activistpost.com/2017/09/u-s-presi...   
11    http://beforeitsnews.com/sports/2017/10/2017-f...   
12    http://beforeitsnews.com/sports/2017/09/just-s...   
13    https://www.reuters.com/article/us-deloitte-cy...   
14    http://beforeitsnews.com/sports/2017/10/107-ch...   
15    https://www.nytimes.com/2017/10/04/us/las-vega... 

In [6]:

df['category_id'] = df['URLs'].factorize()[0]
category_id_df = df[['URLs', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'URLs']].values)
# print(df.head())

In [7]:

##data absorbing done

##feature extraction begins here


##tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2))
features = tfidf.fit_transform(df.Body).toarray()
labels = df.category_id
print(features.shape)


(3988, 68312)


In [8]:

##uni and bi-grams

from sklearn.feature_selection import chi2
import numpy as np
N = 4
for URLs, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  # print("# '{}':".format(Product))
  # print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  # print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))



In [11]:

##first test model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df['Body'], df['URLs'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)


In [12]:
##first test models ends here

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The mi